In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer


In [2]:
def extract_pdf(path) :
    pdf_path = path
    reader = PdfReader(pdf_path)

    # Initialize an empty string to store the extracted text
    extracted_text = ""

    for page in reader.pages:
        extracted_text += page.extract_text()

    print(extracted_text)
    return extracted_text


In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Set your desired chunk size
    chunk_overlap=200  # Set your desired chunk overlap
)

In [4]:
def make_chunks(text) :
    return text_splitter.split_text(text)


In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')


c:\python\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Phone : 7812026222
Sri Eshwar College of Engineering 
Government Boys Higher Secondary School
Government Boys Higher Secondary SchoolGitHub LinkedIn 
Machine Learning Intern - Prodigy InfoTech                                                                                                          2024
Machine Learning (ML) internship , Developed and implemented ML models, conducted training  and
validation, and analyzed performance metrics. Proficient in data preprocessing, model deployment, and  
effective communication within interdisciplinary teams.
MERN Stack - RVTechlearn                                                                                                                                    2024
Interned on MERN stack, gaining hands-on experience in full-stack web development. Developed user-friendly
interfaces with React.js, built RESTful APIs with Node.js and Express.js, and managed MongoDB databases.
Collaborated on version control with Git and contributed to problem-

In [ ]:
# Encode the sentences into embeddings
extracted_txt = extract_pdf("C:/Users/Rajesh/Downloads/Rajesh_Res5.pdf (9).pdf")
chunks = make_chunks(extracted_txt)
embeddings = model.encode(chunks)

print("Sentence embeddings:")
print(embeddings)
vectors = []
for i, vec in enumerate(embeddings):
    vectors.append({"id": f"vec{i+1}", "values": vec.tolist()})

print(vectors)

In [9]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="a7bdaea1-66dd-4525-ae19-6ab0191ff9cb")
index_name = "test1"


In [1]:
index_name

NameError: name 'index_name' is not defined

In [ ]:

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
) 


In [10]:
index = pc.Index(index_name)
# index.upsert( vectors )

In [18]:
def get_context(ques) :
    ques_emb = model.encode(ques)
    DB_response = index.query(
        vector=ques_emb.tolist(),
        top_k=3,
        include_values=True
    )

    cont = ""
    for i in range(len(DB_response['matches'])) :
        cont += chunks[int(DB_response['matches'][i]['id'][3:])-1]
    return cont

In [12]:
from groq import Groq

client = Groq(
    api_key="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",
)

In [21]:
def chat(query) :
    context = get_context(query)
    # print(context)
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Context : {context} , Analyse and understand the above context completely and answer the below query , Query : {query}",
            }
        ],
        model="llama3-8b-8192",
    )

    print(chat_completion.choices[0].message.content)


In [22]:
chat("number of interns completed ?")



KeyboardInterrupt



In [2]:
from flask import Flask, request
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
paths = []


In [6]:
!flask --version

Python 3.10.13
Flask 2.2.2
Werkzeug 2.2.2


In [7]:


@app.route('/upload', methods=['POST'])
def upload_file2():
    print("HIII")
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    if file:
        filename = file.filename
        file.save(os.path.join('C:/NLP/DocChat', filename))
        paths.append(filename)
        print(paths)
        return 'File uploaded successfully'
    


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/May/2024 20:14:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 20:14:57] "POST /upload_files HTTP/1.1" 200 -


Uploaded file path: AI_Syllabus.pdf


In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer


from pinecone import Pinecone, ServerlessSpec


In [2]:
from groq import Groq

client = Groq(
    api_key="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",
)

In [3]:
from flask import Flask, render_template, request, jsonify
import os
paths = []
model = SentenceTransformer('all-MiniLM-L6-v2')
index_name = "final1"
pc = Pinecone(api_key="a7bdaea1-66dd-4525-ae19-6ab0191ff9cb")
index = pc.Index(index_name)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Set your desired chunk size
    chunk_overlap=200  # Set your desired chunk overlap
)
tot_chunks = []
def make_chunks(text) :
    return text_splitter.split_text(text)

app = Flask(__name__)

def get_context(ques) :
    index = pc.Index(index_name)

    ques_emb = model.encode(ques)
    DB_response = index.query(
        vector=ques_emb.tolist(),
        top_k=3,
        include_values=True
    )

    cont = ""
    for i in range(len(DB_response['matches'])) :
        cont += tot_chunks[int(DB_response['matches'][i]['id'][3:])-1]
    return cont

def extract_pdf(path) :
    pdf_path = path
    reader = PdfReader(pdf_path)

    # Initialize an empty string to store the extracted text
    extracted_text = ""

    for page in reader.pages:
        extracted_text += page.extract_text()

    print(extracted_text)
    return extracted_text

@app.route('/')
def index():
    return render_template('index-cpy.html')

@app.route('/upload_files', methods=['POST'])
def upload_file():
    global tot_chunks
    files = request.files.getlist('files')
    for file in files:
        file_path = file.filename
        file.save(file_path)
        paths.append(file_path)
        print(f"Uploaded file path: {file_path}")

    extracted = ""
    for path in paths: 
        extracted += extract_pdf(path)
    tot_chunks = make_chunks(extracted)
    tot_embeddings = model.encode(tot_chunks)

    tot_vectors = []
    for i, vec in enumerate(tot_embeddings):
        tot_vectors.append({"id": f"vec{i+1}", "values": vec.tolist()})

    # pc.create_index(
    #     name=index_name,
    #     dimension=384,
    #     metric="cosine",
    #     spec=ServerlessSpec(
    #         cloud='aws', 
    #         region='us-east-1'
    #     ) 
    # ) 
    index = pc.Index(index_name)
    # index.upsert( tot_vectors )
    return "Success !"


@app.route('/send', methods=['POST'])
def send_text():
    query = request.json['text']
    context = get_context(query)
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Context : {context} , Analyse and understand the above context completely and answer the below query , Query : {query}",
            }
        ],
        model="llama3-8b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    return response_text

c:\python\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [08/May/2024 22:39:22] "GET / HTTP/1.1" 200 -


Uploaded file path: Rajesh_Res5.pdf (10).pdf
Phone : 7812026222
Sri Eshwar College of Engineering 
Government Boys Higher Secondary School
Government Boys Higher Secondary SchoolGitHub LinkedIn 
Machine Learning Intern - Prodigy InfoTech                                                                                                          2024
Machine Learning (ML) internship , Developed and implemented ML models, conducted training  and
validation, and analyzed performance metrics. Proficient in data preprocessing, model deployment, and  
effective communication within interdisciplinary teams.
MERN Stack - RVTechlearn                                                                                                                                    2024
Interned on MERN stack, gaining hands-on experience in full-stack web development. Developed user-friendly
interfaces with React.js, built RESTful APIs with Node.js and Express.js, and managed MongoDB databases.
Collaborated on version

127.0.0.1 - - [08/May/2024 22:39:33] "POST /upload_files HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 22:40:25] "POST /send HTTP/1.1" 200 -
